In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from scipy import sparse

# Reference: https://beckernick.github.io/music_recommender/

# Loading and processing data

In [2]:
# Read in tsv files. 
user_data = pd.read_table('usersha1-artmbid-artname-plays.tsv',
                          header = None, 
                          names = ['users', 'musicbrainz-artist-id', 'artist-name', 'plays'],
                          usecols = ['users', 'artist-name', 'plays'])
user_profiles = pd.read_table('usersha1-profile.tsv',
                          header = None,
                          names = ['users', 'gender', 'age', 'country', 'signup'],
                          usecols = ['users', 'country'])

In [3]:
# Remove artists with no plays. 
if user_data['artist-name'].isnull().sum() > 0:
    user_data = user_data.dropna(axis = 0, subset = ['artist-name'])

In [4]:
# Add column with sum of total artist plays.
artist_plays = (user_data.
     groupby(by = ['artist-name'])['plays'].
     sum().
     reset_index().
     rename(columns = {'plays': 'total_artist_plays'})
     [['artist-name', 'total_artist_plays']]
    )

user_data_with_artist_plays = user_data.merge(artist_plays, left_on = 'artist-name', right_on = 'artist-name', how = 'left')

In [5]:
# Remove artists with less than popularity_threshold plays. 
popularity_threshold = 1000000

user_data_popular_artists = user_data_with_artist_plays.query('total_artist_plays >= @popularity_threshold')
combined = user_data_popular_artists.merge(user_profiles, left_on = 'users', right_on = 'users', how = 'left')

usa_data = combined.query('country == \'United States\'')

In [6]:
# Add column with sum of total user plays. 
user_plays = (usa_data.
     groupby(by = ['users'])['plays'].
     sum().
     reset_index().
     rename(columns = {'plays': 'total_user_plays'})
     [['users', 'total_user_plays']]
    )

plays_data = usa_data.merge(user_plays, left_on = 'users', right_on = 'users', how = 'left')

In [7]:
# Remove users with less than plays_threshold plays. 
plays_threshold = 1000

final_data = plays_data.query('total_user_plays >= @plays_threshold')

# Creating hypergraph

In [8]:
# E: number of users
# V: number of artists
# erase_prob: chance user rating is erased (for UDOA)
def make_hypergraph(E, V, erase_prob, random_seed):
    np.random.seed(random_seed)
    
    max_users = E
    max_artists = V
     
    W = np.zeros((V, E)) # hyperedge-weight matrix, |V|x |E|, each row corresponds to a movie. 
    R = np.zeros((E, V)) # edge-dependent vertex-weight matrix, |E| x |V|, each row corresponds to a user.
    true_R = np.zeros((E, V)) # same but without erasing.

    num_pairs = 0
    
    curr_avail_user_index = 0
    curr_avail_artist_index = 0
    user_dict = {} 
    artist_dict = {}
    
    for _, row in final_data.iterrows():
        user = row['users']
        artist = row['artist-name']
        plays = int(row['plays'])

        user_index = user_dict.get(user)

        if user_index == None:
            if curr_avail_user_index < max_users:
                user_dict[user] = curr_avail_user_index
                user_index = curr_avail_user_index
                curr_avail_user_index += 1
            else:
                continue

        artist_index = artist_dict.get(artist)

        if artist_index == None:
            if curr_avail_artist_index < max_artists:
                artist_dict[artist] = curr_avail_artist_index
                artist_index = curr_avail_artist_index
                curr_avail_artist_index += 1
            else:
                continue

        true_R[user_index][artist_index] = plays

        err = np.random.rand(1)
        if err > erase_prob:
            W[artist_index][user_index] = 1
            R[user_index][artist_index] = plays

            num_pairs += 1
            
    return W, R, true_R, num_pairs

# Similarity metrics

In [9]:
def make_similarities(E, V, R):
    power_similarities = np.ones((E, E))
    dot_similarities = np.ones((E, E))

    for i in range(E):
        for j in range(i+1, E):

            dot_product = np.dot(R[i], R[j])

            if dot_product > 1:
                dot_similarities[i][j] = dot_product
                dot_similarities[j][i] = dot_product  

            for song in range(V):
                if R[i][song] != 0 and R[j][song] != 0:

                    power_similarities[i][j] *= 2
                    power_similarities[j][i] *= 2
                    
    return power_similarities, dot_similarities

In [10]:
# We want the nonzero rows of W and R to sum to 1 
def row_normalize(X):
    Y = np.matrix.copy(X)
    for i in range(len(Y)):
        row = Y[i]
        row_sum = np.sum(row)
        if row_sum != 0:
            Y[i] = Y[i]/row_sum   
    return Y

In [11]:
def get_P(W, R):
    Rs = sparse.csr_matrix(row_normalize(R))
    Ws = sparse.csr_matrix(row_normalize(W))
    P = np.transpose(Ws.dot(Rs))
    return P

In [12]:
# A weight matrix customized for a particular user.
def get_Wi(W, user_index, similarities):
    Wi = np.copy(W)
    for other_user_index in range(len(similarities[user_index])):
        if other_user_index == user_index:
            continue
        similarity = similarities[user_index][other_user_index]
        Wi[:,other_user_index] *= similarity
        
    return Wi

In [13]:
def get_P_hg(Wi, R):
    Rs = sparse.csr_matrix(row_normalize(R))
    Wis = sparse.csr_matrix(row_normalize(Wi))
    Pi = np.transpose(Wis.dot(Rs))
    return Pi

# Creating graph

In [14]:
def get_P_g(Wi, E, V, R, num_pairs):
    
    WiG = np.zeros((V+E, num_pairs)) 
    RG = np.zeros((num_pairs, V+E)) 

    curr_edge_index = 0 

    for i in range(V):
        for j in range(E):
            if R[j][i] != 0:
                # index of movie i in graph = i
                # index of user j in graph = V+j

                RG[curr_edge_index][V+j] = 1
                RG[curr_edge_index][i] = 1

                WiG[V+j][curr_edge_index] = Wi[i][j] * R[j][i]
                WiG[i][curr_edge_index] = WiG[V+j][curr_edge_index]

                curr_edge_index += 1
                
    WiGs = sparse.csr_matrix(row_normalize(WiG))
    RGs = sparse.csr_matrix(row_normalize(RG))
    PiG = np.transpose(WiGs.dot(RGs))
    
    return PiG

# Compute rankings

In [15]:
# given probability transition matrix P
# where P_{v,w} = Prob(w -> v)
# find pagerank scores with restart probability r
def compute_pr(P, r, n, home, eps=1e-8):
    
    x = np.ones(n) / n*1.0

    flag = True
    t=0
        
    while flag:
        x_new = (1-r)*P*x

        x_new = x_new + home * r 
        
        if np.linalg.norm(x_new - x,ord=1) < eps and t > 100:
            flag = False
        t=t+1
        x = x_new
    
    return x

In [16]:
def compute_ranking_hg(user_index, V, R, P, r=0.15):
    
    # personalize the algorithm by restarting at any of the movies a certain user originally watched
    home_hg = np.zeros(V)

    for j in range(V):
        if R[user_index][j] != 0:
            home_hg[j] = 1

    if np.sum(home_hg) > 0:
        home_hg = home_hg / np.sum(home_hg)

    ranking_hg = compute_pr(P, r, V, home_hg).flatten()
        
    return ranking_hg

In [17]:
def compute_ranking_g(user_index, V, E, P, r=0.15):
    
    # restart at user's vertex
    home_g = np.zeros(V+E)
    home_g[V+user_index] = 1

    curr_rankings_g = compute_pr(P, r, V+E, home_g).flatten()
    ranking_g = curr_rankings_g[:V]
        
    return ranking_g

# Evaluate rankings

In [18]:
# Source: https://www.aaai.org/Papers/IJCAI/2007/IJCAI07-444.pdf
def calc_avg_doa(num_movies, given_rating, true_rating, ranking):
    
    total_pairs = 0
    correct_pairs = 0
    
    # All pairs of movies. 
    for i in range(num_movies):
        for j in range(i+1, num_movies):
            if true_rating[i] < true_rating[j]:
                total_pairs += 1
                if ranking[i] < ranking[j]:
                    correct_pairs += 1
            elif true_rating[i] > true_rating[j]:
                total_pairs += 1
                if ranking[i] > ranking[j]:
                    correct_pairs += 1
       
    if total_pairs == 0:
        return -1
    return correct_pairs/total_pairs

def calc_avg_udoa(num_movies, given_rating, true_rating, ranking):
    
    total_pairs = 0
    correct_pairs = 0
    
    # All pairs of movies. 
    for i in range(num_movies):
        for j in range(i+1, num_movies):
            if given_rating[i] == 0 and true_rating[i] != 0 and given_rating[j] == 0 and true_rating[j] != 0:
                if true_rating[i] < true_rating[j]:
                    total_pairs += 1
                    if ranking[i] < ranking[j]:
                        correct_pairs += 1
                elif true_rating[i] > true_rating[j]:
                    total_pairs += 1
                    if ranking[i] > ranking[j]:
                        correct_pairs += 1
       
    if total_pairs == 0:
        return -1
    return correct_pairs/total_pairs

# Code to run things

In [19]:
def run_trials(E, V, W, R, true_R, num_pairs, similarities, is_hg):

    udoa = 0
    ucount = 0
    
    doa = 0
    dcount = 0
    
    for i in range(E):
        
        Wi = get_Wi(W, i, similarities)
        
        if is_hg:
            P = get_P_hg(Wi, R)
            ranking = compute_ranking_hg(i, V, R, P)
        else:
            P = get_P_g(Wi, E, V, R, num_pairs)
            ranking = compute_ranking_g(i, V, E, P)

        curr_udoa = calc_avg_udoa(V, R[i], true_R[i], ranking)
        curr_doa = calc_avg_doa(V, R[i], true_R[i], ranking)

        if curr_udoa > -1:
            udoa += curr_udoa
            ucount += 1
        if curr_doa > -1:
            doa += curr_doa
            dcount += 1
    
    if ucount != 0:
        udoa = udoa/ucount
    else:
        udoa = -1
        
    if dcount != 0:
        doa = doa/dcount
    else:
        doa = -1
        
    return doa, udoa

In [20]:
def run_dumb_trials_hg(E, V, R, true_R, P):

    udoa = 0
    ucount = 0
    
    doa = 0
    dcount = 0
    
    for i in range(E):
        
        ranking = compute_ranking_hg(i, V, R, P)
        curr_udoa = calc_avg_udoa(V, R[i], true_R[i], ranking) 
        curr_doa = calc_avg_doa(V, R[i], true_R[i], ranking)

        if curr_udoa > -1:
            udoa += curr_udoa
            ucount += 1
            
        if curr_doa > -1:
            doa += curr_doa
            dcount += 1

    if ucount != 0:
        udoa = udoa/ucount
    else:
        udoa = -1
        
    if dcount != 0:
        doa = doa/dcount
    else:
        doa = -1
        
    return doa, udoa

In [21]:
def run_dumb_trials_g(E, V, R, true_R, P):

    udoa = 0
    ucount = 0
    
    doa = 0
    dcount = 0
    
    for i in range(E):
        
        ranking = compute_ranking_g(i, V, E, P)
        curr_udoa = calc_avg_udoa(V, R[i], true_R[i], ranking) 
        curr_doa = calc_avg_doa(V, R[i], true_R[i], ranking)

        if curr_udoa > -1:
            udoa += curr_udoa
            ucount += 1
            
        if curr_doa > -1:
            doa += curr_doa
            dcount += 1

    if ucount != 0:
        udoa = udoa/ucount
    else:
        udoa = -1
        
    if dcount != 0:
        doa = doa/dcount
    else:
        doa = -1
        
    return doa, udoa

In [22]:
def do_everything(E, V, erase_prob, random_seed):
    print("erase_prob=%.3f, random_seed=%d" % (erase_prob, random_seed))

    W, R, true_R, num_pairs = make_hypergraph(E, V, erase_prob, random_seed)
    print("made hypergraph")
    
    array = [0 for _ in range(7)] 
    power_sims, dot_sims = make_similarities(E, V, R)
    P = get_P(W, R)
    Pg = get_P_g(W, E, V, R, num_pairs)
    
    hgdumb, uhgdumb = run_dumb_trials_hg(E, V, R, true_R, P)
    gdumb, ugdumb = run_dumb_trials_g(E, V, R, true_R, Pg)
    print("finished trial 0")
    
    hg1, uhg1 = run_trials(E, V, W, R, true_R, num_pairs, power_sims, True)
    g1, ug1 = run_trials(E, V, W, R, true_R, num_pairs, power_sims, False)
    print("finished trial 1")
    
    hg2, uhg2 = run_trials(E, V, W, R, true_R, num_pairs, dot_sims, True)
    g2, ug2 = run_trials(E, V, W, R, true_R, num_pairs, dot_sims, False)
    print("finished trial 2")
    
    #hg3, uhg3 = run_trials(E, V, W, R, true_R, num_pairs, log_dot_sims, True)
    #g3, ug3 = run_trials(E, V, W, R, true_R, num_pairs, log_dot_sims, False)
    #print("finished trial 3")

    return [hgdumb, uhgdumb, gdumb, ugdumb, hg1, uhg1, g1, ug1, hg2, uhg2, g2, ug2]

In [23]:
def do_everything_n_times(E, V, erase_prob, n):
    
    avgs = [0 for _ in range(16)]
    
    for i in range(n):
        udoas = do_everything(E, V, erase_prob, i)
        avgs = np.add(avgs, udoas)
        print()
        
    return np.divide(avgs, n)

# Actually running things

In [24]:
array = do_everything_n_times(100, 500, 0.15, 5)

erase_prob=0.150, random_seed=0
made hypergraph
finished trial 0
finished trial 1
finished trial 2
finished trial 3

erase_prob=0.150, random_seed=1
made hypergraph
finished trial 0
finished trial 1
finished trial 2
finished trial 3

erase_prob=0.150, random_seed=2
made hypergraph
finished trial 0
finished trial 1
finished trial 2
finished trial 3

erase_prob=0.150, random_seed=3
made hypergraph
finished trial 0
finished trial 1
finished trial 2
finished trial 3

erase_prob=0.150, random_seed=4
made hypergraph
finished trial 0
finished trial 1
finished trial 2
finished trial 3



In [27]:
print("dumb hypergraph doa:", array[0])
print("dumb hypergraph udoa:", array[1])
print("dumb graph doa:", array[2])
print("dumb graph udoa:", array[3])
print()

print("power hypergraph doa:", array[4])
print("power hypergraph udoa:", array[5])
print("power graph doa:", array[6])
print("power graph udoa:", array[7])
print()

print("dot hypergraph doa:", array[8])
print("dot hypergraph udoa:", array[9])
print("dot graph doa:", array[10])
print("dot graph udoa:", array[11])

#print("log-dot hypergraph doa:", array[12])
#print("log-dot hypergraph udoa:", array[13])
#print("log-dot graph doa:", array[14])
#print("log-dot graph udoa:", array[15])

dumb hypergraph doa: 0.9391555764739387
dumb hypergraph udoa: 0.5278802584066591
dumb graph doa: 0.9493473418399075
dumb graph udoa: 0.5299835586804126

power hypergraph doa: 0.9350590906964685
power hypergraph udoa: 0.5293953240665242
power graph doa: 0.9496647510289915
power graph udoa: 0.5434046037834884

dot hypergraph doa: 0.9367001736669389
dot hypergraph udoa: 0.5205203618266889
dot graph doa: 0.9477262268848042
dot graph udoa: 0.5280138927767432
